<a href="https://colab.research.google.com/github/mtoce/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/Assig_3_LS_DS_223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


# Pre-Model Prep

### Setup

You can work locally (follow the [local setup instructions](https://lambdaschool.github.io/ds/unit2/local/)) or on Colab (run the code cell below).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# New code for Pandas Profiling version 2.4
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train, minimal=True).to_notebook_iframe()

# profile

## Train / Validate / Test Split

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [4]:
train, val = train_test_split(train, train_size=0.80, test_size=0.20, stratify=train['status_group'], random_state=17)
train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

## Wrangle Function

In [0]:
import numpy as np

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # replace population=1 with np.NaN
    X['population'] = X['population'].replace({1: np.NaN})
    # return the wrangled dataframe
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
#train.dtypes

In [7]:
train.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,longitude_MISSING,latitude_MISSING,construction_year_MISSING,gps_height_MISSING,population_MISSING
43465,1000.0,2013-03-09,Roman Cathoric Same,1687.0,Roman Cathoric Same,37.631842,-3.725220,Ngujini Sec School,0,Pangani,Kwabmnaro,Kilimanjaro,3,2,Mwanga,Ngujini,40.0,True,VWC,Kindoroko water supply,False,2004.0,gravity,gravity,gravity,vwc,user-group,pay monthly,soft,good,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,functional,False,False,False,False,False
20252,500.0,2011-11-04,Nado,1493.0,wananchi,34.639256,-8.970386,Kwa Elai Mkahala,0,Rufiji,Igando,Iringa,11,4,Njombe,Usuka,85.0,True,WUA,wanging'ombe water supply s,True,2003.0,gravity,gravity,gravity,wua,user-group,pay monthly,soft,good,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional,False,False,False,False,False
3674,0.0,2013-02-27,Danida,859.0,DANIDA,36.015272,-10.753528,Kwa Fuko,0,Ruvuma / Southern Coast,Kidalini,Ruvuma,10,5,Namtumbo,Mkongo,100.0,True,VWC,Njalamatata water gravity scheme,True,NaN,gravity,gravity,gravity,vwc,user-group,pay when scheme fails,soft,good,dry,unknown,other,unknown,communal standpipe,communal standpipe,non functional,False,False,True,False,False
15441,0.0,2011-03-22,Unhcr,341.0,Handeni Trunk Main(,38.526482,-5.609702,Kwa Semboga,0,Pangani,Bulumusa,Tanga,4,6,Handeni,Mgambo,NaN,True,VWC,Handeni Trunk Main(H,True,2003.0,submersible,submersible,submersible,vwc,user-group,never pay,soft,good,insufficient,river,river/lake,surface,communal standpipe multiple,communal standpipe,non functional,False,False,False,False,False
41367,0.0,2011-03-13,Water,NaN,Commu,36.529519,-6.183249,Shuleni,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Ugogoni,NaN,True,VWC,Msin,False,NaN,mono,mono,motorpump,vwc,user-group,pay per bucket,salty,salty,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,False,False,True,True,True


In [0]:
# cardinality

## Reduce Cardinality for HC Features

In [0]:
def ReduceCardinality(X):
  '''Reduces Cardinality for features to make them model-relevant
  X: input DF
  n: cardinality of features for model
  '''

  X = X.copy()
  # Reduce cardinality of specific features

  # date_recorded: change to just year
  X['date_recorded'] = pd.to_datetime(X['date_recorded']).dt.year

  # Quickly impute just construction_year column since I we need for new features
  # X['construction_year'] = X['construction_year'].fillna(value=X['construction_year'].median())
#-------------------------------------------------------------------------------
  # Feature Engineering
  X['time_til_inspection'] = X['date_recorded'] - X['construction_year']

#-------------------------------------------------------------------------------
  # keep top 24 values and replace 25th with other
  # df.apply(lambda x: x == 'OTHER' if x.name == ['subvillage', 'funder', 'installer', 'wpt_name', 'lga', 'scheme_name'] else )

  # subvillage: keep top 24
  top1 = X['subvillage'].value_counts()[:24].index
  X.loc[~X['subvillage'].isin(top1), 'subvillage'] = 'OTHER'
  
  # funder: keep top 25
  top2 = X['funder'].value_counts()[:24].index
  X.loc[~X['funder'].isin(top2), 'funder'] = 'OTHER'

  # installer: keep top 25
  top3 = X['installer'].value_counts()[:24].index
  X.loc[~X['installer'].isin(top3), 'installer'] = 'OTHER'

  # wpt_name: keep top 15
  top4 = X['wpt_name'].value_counts()[:24].index
  X.loc[~X['wpt_name'].isin(top4), 'wpt_name'] = 'OTHER'

  # lga: keep top 25
  top5 = X['lga'].value_counts()[:24].index
  X.loc[~X['lga'].isin(top5), 'lga'] = 'OTHER'

  # ward: keep top 25
  top6 = X['ward'].value_counts()[:24].index
  X.loc[~X['ward'].isin(top6), 'ward'] = 'OTHER'

  # scheme_name: keep top 25
  top7 = X['scheme_name'].value_counts()[:24].index
  X.loc[~X['scheme_name'].isin(top7), 'scheme_name'] = 'OTHER'

  return X

In [0]:
# train = ReduceCardinality(train)
# val = ReduceCardinality(val)
# test = ReduceCardinality(test)

## Manual Impute Function

In [0]:
def ComplexImputer(X):
  '''Imputes NaN values in multiple different ways depending on the column'''

  X = X.copy()

  # Column lists for different imputer methods
  cols_mean = ['gps_height', 'latitude', 'longitude']
  cols_median = ['construction_year', 'population', 'public_meeting']
  cols_mode = ['permit', 'scheme_management']

  # replace these columns' NaNs with the mean of column
  X[cols_mean] = X[cols_mean].fillna(value=X[cols_mean].mean())
  
  # replace these columns' NaNs with median
  X[cols_median] = X[cols_median].fillna(value=X[cols_median].median())

  # replace these columns' NaNs with mode
  X['permit'] = X['permit'].fillna(value=True)
  X['scheme_management'] = X['scheme_management'].fillna(value='VWC')

  return X

# Final Function to Run on Data before we send into Pipeline

In [0]:
def Wrangle2(X):
  """Wrangle train, validate, and test sets in the same way"""

  X = X.copy()

  # run our manual imputer & cardinality functions
  X = ComplexImputer(X)
  X = ReduceCardinality(X)

  return X

In [0]:
train = Wrangle2(train)
val = Wrangle2(val)
test = Wrangle2(test)

In [44]:
train.isnull().sum()

amount_tsh                   0
date_recorded                0
funder                       0
gps_height                   0
installer                    0
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                   0
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting               0
scheme_management            0
scheme_name                  0
permit                       0
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
water_quality                0
quality_group                0
quantity                     0
source                       0
source_t

## Get Baseline

In [0]:
# define target
target = 'status_group'

In [46]:
# select baseline - categorical - which one is the mode? functional: 54% is the baseline
train['status_group'].value_counts(normalize=True)

functional                 0.543077
non functional             0.384238
functional needs repair    0.072685
Name: status_group, dtype: float64

In [0]:
train_features = train.drop([target], axis=1)
numeric_features = train_features.select_dtypes(include='number').columns.tolist()
cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = cardinality[cardinality <= 25].index.tolist()

In [0]:
features = numeric_features + categorical_features

In [0]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

# Make Pipeline

In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt

In [51]:
%%time
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    StandardScaler(),
    RandomForestClassifier(n_jobs=-1, random_state=17)
)
# run pipeline fit
pipeline.fit(X_train, y_train)

# Predict on test
y_pred = pipeline.predict(X_test)

print('Train Accuracy: ', pipeline.score(X_train, y_train))
print('Validation Accuracy: ', pipeline.score(X_val, y_val))

Train Accuracy:  0.9974957912457912
Validation Accuracy:  0.8046296296296296
CPU times: user 16.7 s, sys: 271 ms, total: 17 s
Wall time: 9.59 s


In [52]:
# declare parameter distributions and run randomized search cross-validation
param_distributions = {
    'randomforestclassifier__n_estimators': [50, 75, 100, 125, 150, 175, 200],
    'randomforestclassifier__max_depth': [16, 18, 20, 22, 24, 26],
    'randomforestclassifier__min_samples_split': [None, 2, 4, 6, 8],
    'randomforestclassifier__criterion': ['gini', 'entropy'],
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  6.1min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=['funder',
                                                                   'installer',
                                                                   'wpt_name',
                                                                   'basin',
                                                                   'subvillage',
                                                                   'region',
                                                                   'lga',
                                                                   'ward',
                                                                   'public_meeting',
                                                                   'scheme_management',
                                                   

In [58]:
print('Best hyperparameters: ', search.best_params_)
print('Cross-validation Accuracy: ', search.best_score_)

#n_estimators=100, min_samples_split=4, criterion='entropy'
pipeline_best = search.best_estimator_

Best hyperparameters:  {'randomforestclassifier__n_estimators': 100, 'randomforestclassifier__min_samples_split': 4, 'randomforestclassifier__max_depth': 22, 'randomforestclassifier__criterion': 'entropy'}
Cross-validation Accuracy:  0.8073442760942762


In [59]:
pipeline_best.fit(X_train, y_train)

y_pred = pipeline_best.predict(X_test)

print('Pipeline-best Train Accuracy: ', pipeline_best.score(X_train, y_train))
print('Pipeline-best Validation Accuracy: ', pipeline_best.score(X_val, y_val))

Pipeline-best Train Accuracy:  0.9466329966329966
Pipeline-best Validation Accuracy:  0.8073232323232323


## Plot Feature Importances

In [0]:
# model = pipeline.named_steps['randomforestclassifier']
# encoder = pipeline.named_steps['ordinalencoder']
# encoded_columns = encoder.transform(X_val).columns
# importances = pd.Series(model.feature_importances_, encoded_columns)
# plt.figure(figsize=(10,30))
# importances.sort_values().plot.barh();

## Save to .csv and submit to Kaggle

In [0]:
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission.csv', index=False)